In [1]:
#you'll need to pip install a few things (again this is in linux but it shouldn't be too hard to install these packages in mac)
# !pip install mido
# !pip install midi2audio
# !sudo apt-get install fluidsynth

In [2]:
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage, bpm2tempo, tempo2bpm, second2tick, tick2second
from midi2audio import FluidSynth
from IPython.display import Audio
import os

In [3]:
#this generates the note name from the midi note number (aka note 60 is C4, 61 is C#4, etc.)

NOTE_NAMES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

def get_note_name(midi_note):
    """Get the note name from a MIDI note number."""
    return NOTE_NAMES[midi_note % 12] + str(midi_note // 12 - 1)  # Corrected the octave offset


In [4]:
#this function constructs the chord structure for each type of chord named.

def generate_chords(start, end, chord_type='', num_notes=3):
    if chord_type not in ['major', 'minor', 'dom7', 'diminished', 'augmented', 'sus2', 'sus4']:
        raise ValueError('Invalid chord type. Must be one of "major", "minor", "dom7", "diminished", "augmented", "sus2", "sus4".')

    chords = []
    for root in range(start, end+1):
        if chord_type == 'major':
            chord = [root, root + 4, root + 7]  # Major triad
            if num_notes >= 4:
                chord.append(root + 11)  # Major seventh
            if num_notes >= 5:
                chord.append(root + 14)  # Major ninth
        elif chord_type == 'minor':
            chord = [root, root + 3, root + 7]  # Minor triad
            if num_notes >= 4:
                chord.append(root + 10)  # Minor seventh
            if num_notes >= 5:
                chord.append(root + 14)  # Minor ninth
        elif chord_type == 'dom7':
            chord = [root, root + 4, root + 7, root + 10]  # Dominant 7th
            if num_notes >= 5:
                chord.append(root + 14)  # Dominant 9th
        elif chord_type == 'diminished':
            chord = [root, root + 3, root + 6]
        elif chord_type == 'augmented':
            chord = [root, root + 4, root + 8]
        elif chord_type == 'sus2':
            chord = [root, root + 2, root + 7]
        elif chord_type == 'sus4':
            chord = [root, root + 5, root + 7]
        chords.append(chord)
    return chords

In [5]:
start_note = 24  # C1
end_note = 96  # C7

#this is the dictionary of all the chords we want to generate. The key is the name of the chord, and the value is the chord structure.
chord_types = {
    'major': [3, 4],
    'minor': [3, 4],
    'dom7': [4, 5],
    'diminished': [3],
    'augmented': [3],
    'sus2': [3],
    'sus4': [3]
}

chords = {}

for chord_type, num_notes_list in chord_types.items():
    for num_notes in num_notes_list:
        key = f"{chord_type}_{'triads' if num_notes == 3 else 'sevenths' if num_notes == 4 else 'ninths'}"
        chords[key] = generate_chords(start_note, end_note, chord_type=chord_type, num_notes=num_notes)

# Now you can access any chord using the chords dictionary. For example:
print(chords['major_triads'])
print(chords['minor_sevenths'])

[[24, 28, 31], [25, 29, 32], [26, 30, 33], [27, 31, 34], [28, 32, 35], [29, 33, 36], [30, 34, 37], [31, 35, 38], [32, 36, 39], [33, 37, 40], [34, 38, 41], [35, 39, 42], [36, 40, 43], [37, 41, 44], [38, 42, 45], [39, 43, 46], [40, 44, 47], [41, 45, 48], [42, 46, 49], [43, 47, 50], [44, 48, 51], [45, 49, 52], [46, 50, 53], [47, 51, 54], [48, 52, 55], [49, 53, 56], [50, 54, 57], [51, 55, 58], [52, 56, 59], [53, 57, 60], [54, 58, 61], [55, 59, 62], [56, 60, 63], [57, 61, 64], [58, 62, 65], [59, 63, 66], [60, 64, 67], [61, 65, 68], [62, 66, 69], [63, 67, 70], [64, 68, 71], [65, 69, 72], [66, 70, 73], [67, 71, 74], [68, 72, 75], [69, 73, 76], [70, 74, 77], [71, 75, 78], [72, 76, 79], [73, 77, 80], [74, 78, 81], [75, 79, 82], [76, 80, 83], [77, 81, 84], [78, 82, 85], [79, 83, 86], [80, 84, 87], [81, 85, 88], [82, 86, 89], [83, 87, 90], [84, 88, 91], [85, 89, 92], [86, 90, 93], [87, 91, 94], [88, 92, 95], [89, 93, 96], [90, 94, 97], [91, 95, 98], [92, 96, 99], [93, 97, 100], [94, 98, 101], [95

### For this part of the code you'll need to download something called a soundfont. The soundfont is a file that contains the sounds of the piano. The soundfont is used to generate the audio files.You can find them free online. 

I provide links to the soundfonts used here in the README of the dataset. Hopefully it doesn't give you a virus haha!

In [6]:
def play_chord(fs,chord, chord_name, directory=''):
    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)

    for i, chord in enumerate(chord):
        # Create a new MIDI file
        mid = MidiFile()
        track = MidiTrack()
        mid.tracks.append(track)

        # Change to a different instrument (0 = Acoustic Grand Piano)
        track.append(Message('program_change', program=0, time=0))

        # Add the notes of the chord
        for note in chord:
            track.append(Message('note_on', note=note, velocity=64, time=0))
        track.append(Message('note_off', note=note, velocity=64, time=1920))
        for note in chord:
            track.append(Message('note_off', note=note, velocity=64, time=0))

        # Extract the root note name and its MIDI number
        root_note_name = get_note_name(chord[0])
        root_midi_number = chord[0]

        # Save the MIDI file with a descriptive name
        midi_filename = os.path.join(directory, f'{chord_name}_{root_note_name}_{root_midi_number}.mid')
        mid.save(midi_filename)

        # Convert the MIDI file to a WAV file
        wav_filename = os.path.join(directory, f'{chord_name}_{root_note_name}_{root_midi_number}.wav')
        fs.midi_to_audio(midi_filename, wav_filename)

        # Clean up the MIDI file
        os.remove(midi_filename)



In [7]:
# Define the soundfonts and their paths. The name is important because it'll be used in the file names.
soundfonts = {
    'ES100':    '/home/dofo/Downloads/soundfonts/_GD__Kawai_ES100_Concert_Grand_1__V3_/[GD] Kawai ES100 Concert Grand 1 [V3].sf2',
    'D274' :    '/home/dofo/Downloads/soundfonts/_GD__Steinway_Model_D274 (1)',
    'Nord' :    '/home/dofo/Downloads/soundfonts/NordGrand1/NordGrand1.sf2',
    'MODX8' :   '/home/dofo/Downloads/soundfonts/Yamaha_MODX8_piano_fix/Yamaha MODX8 piano.sf2',
    # Add other soundfonts here as needed
}

# Define the chord types, their variable names, directory names, and chord names
chord_definitions = {
    'major_triads': {
        'var_name': 'major_triads', #this is the variable name of the chord type in the chords dictionary
        'dir_name': 'major_triad', #this is the directory name where the chords will be saved
        'chord_name': 'majortriad' #this is what it'll be named in the directory (IMPORTANT FOR LABELING, so be sure to give it a nice name :D)
    },
    'major_sevenths': {
        'var_name': 'major_sevenths',
        'dir_name': 'major_seventh',
        'chord_name': 'majorseventh'
    },

    'minor_triads': {
        'var_name': 'minor_triads',
        'dir_name': 'minor_triad',
        'chord_name': 'minortriad'
    },

    'minor_sevenths': {
        'var_name': 'minor_sevenths',
        'dir_name': 'minor_seventh',
        'chord_name': 'minorseventh'
    },

    'dom7_sevenths': {
        'var_name': 'dom7_sevenths',
        'dir_name': 'dom7_seventh',
        'chord_name': 'dom7seventh'
    },

    'dom7_ninths': {
        'var_name': 'dom7_ninths',
        'dir_name': 'dom7_ninth',
        'chord_name': 'dom7ninth'
    },

    'diminished_triads': {
        'var_name': 'diminished_triads',
        'dir_name': 'diminished_triad',
        'chord_name': 'diminishedtriad'
    },

    'augmented_triads': {
        'var_name': 'augmented_triads',
        'dir_name': 'augmented_triad',
        'chord_name': 'augmentedtriad'
    },

    'sus2_triads': {
        'var_name': 'sus2_triads',
        'dir_name': 'sus2_triad',
        'chord_name': 'sus2triad'
    },

    'sus4_triads': {
        'var_name': 'sus4_triads',
        'dir_name': 'sus4_triad',
        'chord_name': 'sus4triad'
    },
    # Add other chord types here as needed
}


#This saves the chords to the directory specified in the chord_definitions dictionary. Don't save the data to github because it'll take up a lot of space.
# Get the directory of the current script
script_directory = os.path.dirname(os.path.abspath('/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/piano_chord_gen.ipynb'))

# Set the base directory relative to the script's directory. Hopefully this works for u too
base_directory = os.path.join(script_directory, 'dataset', 'chords')

In [8]:
for sf_name, sf_path in soundfonts.items():
    # Expand the ~ to the full path of the user's home directory and create FluidSynth instance
    fs_instance = FluidSynth(os.path.expanduser(sf_path))
    
    for chord_key, chord_info in chord_definitions.items():
        chord_var = chords[chord_info['var_name']]
        directory = os.path.join(base_directory, chord_info['dir_name'])
        chord_name = f"{sf_name}_{chord_info['chord_name']}"
        
        play_chord(fs_instance, chord_var, chord_name, directory)

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords/major_triad/ES100_majortriad_C1_24.wav'..


fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords/major_triad/ES100_majortriad_C#1_25.wav'..


fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


...

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords/sus4_triad/MODX8_sus4triad_E6_88.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords/sus4_triad/MODX8_sus4triad_F6_89.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords/sus4_triad/MODX8_sus4triad_F#6_90.wav'..


fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords/sus4_triad/MODX8_sus4triad_G6_91.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords/sus4_triad/MODX8_sus4triad_G#6_92.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords/sus4_triad/MODX8_sus4triad_A6_93.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2

fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
